In [1]:
%matplotlib inline
import os
import random
#import Pickle as pickle
import numpy as np
import matplotlib.pyplot
from matplotlib.pyplot import imshow
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
from sklearn.decomposition import PCA
from scipy.spatial import distance
#from tqdm import tqdm

Using TensorFlow backend.


In [2]:
model = keras.applications.VGG16(weights='imagenet', include_top=True)
#model.summary()

In [3]:
# get_image will return a handle to the image itself, and a numpy array of its pixels to input the network
def get_image(path):
    img = image.load_img(path, target_size=model.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [4]:
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
#feat_extractor.summary()

In [5]:
images_path = '/home/pyimagesearch/Desktop/stamp/dataset/stamp/'
max_num_images = 100

images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
if max_num_images < len(images):
    images = [images[i] for i in sorted(random.sample(range(len(images)), max_num_images))]

print("keeping %d images to analyze" % len(images))

keeping 100 images to analyze


In [6]:
features = []
for image_path in (images):
    img, x = get_image(image_path);
    feat = feat_extractor.predict(x)[0]
    features.append(feat)

In [7]:
len(features)

100

In [8]:
images_path = '/home/pyimagesearch/Desktop/stamp/dataset/non_stamp/'
max_num_images = 150

images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
if max_num_images < len(images):
    images = [images[i] for i in sorted(random.sample(range(len(images)), max_num_images))]

print("keeping %d images to analyze" % len(images))

keeping 150 images to analyze


In [9]:
for image_path in (images):
    img, x = get_image(image_path);
    feat = feat_extractor.predict(x)[0]
    features.append(feat)

In [10]:
len(features)

250

In [11]:
features = np.array(features)
#pca = PCA(n_components=10)
#pca.fit(features)
#pca_features = pca.transform(features)

In [12]:
X = features #pca_features
y1 = [1]*100
y2 = [0]*150
y = y1 +y2

In [13]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
clf_sign = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf_sign, X, y, cv=5)
clf_sign.fit(X,y)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
scores

array([ 0.96,  0.98,  0.96,  0.96,  0.98])

In [15]:
clf_sign.predict([X[159]])

array([0])

In [16]:
from sklearn.externals import joblib
joblib.dump(clf_sign, 'clf_stamp.pkl')

['clf_stamp.pkl']